In [9]:
#!/usr/bin/env python3.7  
# -*- coding: utf-8 -*- 
#----------------------------------------------------------------------------
# Created By  : name_of_the_creator Hugo Andres Dorado   
# Created Date: 17/02/2022
# version ='1.0'

from msilib import sequence
import pandas as pd
import numpy as np
import random as rd
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from joblib import dump, load
import csv
import re
from Optimization_funs import *

In [10]:

# Read dataset

dataset = pd.read_csv("../data/Cordoba-id-recover.csv",index_col=0)

del dataset["year_sems"]

dataset.columns

X = dataset.copy()

del X["Yield"]

Y = dataset["Yield"]

In [11]:
# Extract ranges

clasVars = pd.read_csv("../data/header_dataset.csv")

In [12]:
# Subset of variables

managment_vars = clasVars[clasVars.Type=="M"]["Variable"].reset_index()["Variable"]
scales_managment_vars = clasVars[clasVars.Type=="M"]["Scale"].reset_index()["Scale"]

other_variables = clasVars[clasVars.Type!="M" ]
other_variables = other_variables[other_variables.Type != "O"]["Variable"].reset_index()["Variable"]

In [13]:
# Split dataset

mat_M = dataset[managment_vars]
mat_O = dataset[other_variables]

In [14]:
# Validation

ds_ranges = AllRangGen(mat_M,scales_managment_vars,managment_vars)


In [15]:
# Persistence

ct = load('../data/pipe_line.joblib')

rf = load('../data/rf_cordoba.joblib') 

In [29]:
row_by_id = dataset.iloc[:, 0] = 295
#df_male = df.loc[is_male]
gen = dataset.loc[[row_by_id]]

gen

,Sowing_Seeds_Number,Seeds_Per_Site,Plant_Density_20_days,Chemical_Treat_Disease,Chemical_Treat_Weeds,Chemical_Treat_Pests,Total_N,Total_P,Total_K,Number_Chemical_Ferti,...,Cultivar,Former_Crop,Field_Drainage,Harvest_Method,Cultivar_Type,Soil_Structure,Runoff,Soil_Texture,Organic_Matter_Content,Yield
ID,,,,,,,,,,,,,,,,,,,,,
295,295,1.0,68000.0,0.0,2.0,1.0,0.092,0.0,0.0,2.0,...,FNC 114,Maiz,NO,Manual,Hibrido,MASIVA,NINGUNO,FrL,MEDIA,4.0


In [33]:
solution = bestGlobHS(fv=gen[other_variables],hms=5,hmcr=0.85,par=0.3,maxNumInp=100,fitnessfun=fitnessfun,
namesds=managment_vars,fixedVars=other_variables,model_train=rf,ranges=ds_ranges,scales=scales_managment_vars,
transf_fun=ct)

d:\OneDrive - CGIAR\Documents\GitHub\eia_climate_service_mockup\src\eia-service\model\Optimization_funs.py:153: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  popfv = fv.append([fv]*(hms-1),ignore_index=True)
d:\OneDrive - CGIAR\Documents\GitHub\eia_climate_service_mockup\src\eia-service\model\Optimization_funs.py:172: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hm = hm.append(newHM)
d:\OneDrive - CGIAR\Documents\GitHub\eia_climate_service_mockup\src\eia-service\model\Optimization_funs.py:183: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_sol[j] = besthm["Performance"].iloc[0]
d:\OneDrive - CGIAR\Documents\GitHub\eia_clim

In [34]:
sol_gen = solution[1]
#del sol_gen["Variable"]
sol_gen.index = gen.index
sol_gen

Variable,Sowing_Seeds_Number,Seeds_Per_Site,Plant_Density_20_days,Chemical_Treat_Disease,Chemical_Treat_Weeds,Chemical_Treat_Pests,Total_N,Total_P,Total_K,Number_Chemical_Ferti,Sowing_Method,Seeds_Treatment,Cultivar,Harvest_Method,Cultivar_Type,Performance
ID,,,,,,,,,,,,,,,,
295,51875.162081,1.479022,79973.468793,3,4,6,0.136297,0.047217,0.032476,4,Mecanizado,SI,DK 1040,Mecanizada,OGM,5.11969


In [19]:
solution[0]

3     4.642272
2     4.662354
0     4.714392
4     4.812420
1     4.987790
        ...   
95    5.126560
96    5.127760
97    5.127760
98    5.127760
99    5.127760
Name: Performance, Length: 100, dtype: float64